# AAI540 - Module 3 Assignment

Victor Hugo Germano

In [1]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:


original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

In [4]:
# S3 definitino
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-housing"

print(default_s3_bucket_name)

sagemaker-us-east-1-440542329720


In [5]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

arn:aws:iam::440542329720:role/LabRole


## Load Dataset to S3

In [43]:
# define s3 address

s3_private_path_data = "s3://{}/housing-m3/csv".format(default_s3_bucket_name)


print(s3_private_path_data)
%store s3_private_path_data


housing_local_file_path = "housing.csv"
housing_s3_object_key = 'housing-m3.csv'


gmaps_local_file_path = "housing_gmaps_data_raw.csv"
gmaps_s3_object_key = 'gmaps-m3.csv'

%store housing_local_file_path
%store gmaps_local_file_path
%store housing_s3_object_key
%store gmaps_s3_object_key

# copy csv to s3
!aws s3 cp $housing_local_file_path $s3_private_path_data/$housing_s3_object_key 
!aws s3 cp $gmaps_local_file_path $s3_private_path_data/$gmaps_s3_object_key 

print(s3_private_path_data)
!aws s3 ls $s3_private_path_data/
    

s3://sagemaker-us-east-1-440542329720/housing-m3/csv
Stored 's3_private_path_data' (str)
Stored 'housing_local_file_path' (str)
Stored 'gmaps_local_file_path' (str)
Stored 'housing_s3_object_key' (str)
Stored 'gmaps_s3_object_key' (str)
upload: ./housing.csv to s3://sagemaker-us-east-1-440542329720/housing-m3/csv/housing-m3.csv
upload: ./housing_gmaps_data_raw.csv to s3://sagemaker-us-east-1-440542329720/housing-m3/csv/gmaps-m3.csv
s3://sagemaker-us-east-1-440542329720/housing-m3/csv
2026-01-26 17:35:38    2057894 gmaps-m3.csv
2026-01-26 17:35:37    1423529 housing-m3.csv


In [44]:


s3_client = boto3.client("s3", region_name=region)

housing_data_object = s3_client.get_object(
    Bucket=default_s3_bucket_name, Key=f"housing-m3/csv/{housing_s3_object_key}"
)
gmaps_data_object = s3_client.get_object(
    Bucket=default_s3_bucket_name, Key=f"housing-m3/csv/{gmaps_s3_object_key}"
)

housing_data = pd.read_csv(io.BytesIO(housing_data_object["Body"].read()))
housing_data["index"] = housing_data.index
gmaps_data = pd.read_csv(io.BytesIO(gmaps_data_object["Body"].read()))
gmaps_data["index"] = gmaps_data.index


In [45]:
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,index
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,4


In [46]:
gmaps_data.head()

,street_number,route,locality-political,administrative_area_level_2-political,administrative_area_level_1-political,country-political,postal_code,address,longitude,latitude,...,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest,index
0,3130,Grizzly Peak Boulevard,Berkeley,Alameda County,California,United States,94705.0,"3130 Grizzly Peak Blvd, Berkeley, CA 94705, USA",-122.23,37.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2005,Tunnel Road,Oakland,Alameda County,California,United States,94611.0,"2005 Tunnel Rd, Oakland, CA 94611, USA",-122.22,37.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,6886,Chabot Road,Oakland,Alameda County,California,United States,94618.0,"6886 Chabot Rd, Oakland, CA 94618, USA",-122.24,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,6365,Florio Street,Oakland,Alameda County,California,United States,94618.0,"6365 Florio St, Oakland, CA 94618, USA",-122.25,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,5407,Bryant Avenue,Oakland,Alameda County,California,United States,94618.0,"5407 Bryant Ave, Oakland, CA 94618, USA",-122.25,37.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


## Feature engineering

In [47]:
# Merge datasets on latitude and longitude
# Merge housing data with gmaps data to get neighborhood information
merged_data = pd.merge(
    housing_data,
    gmaps_data,
    on="index",
    how='left'
)

print(f"Original housing data shape: {housing_data.shape}")
print(f"Merged data shape: {merged_data.shape}")
merged_data.head()

Original housing data shape: (20640, 11)
Merged data shape: (20640, 41)


,longitude_x,latitude_x,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,...,establishment-natural_feature,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# One-hot encode ocean_proximity
# Create one-hot encoded columns for ocean_proximity
ocean_proximity_dummies = pd.get_dummies(merged_data['ocean_proximity'], prefix='', prefix_sep='', dtype=int)

expected_columns = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
for col in expected_columns:
    if col not in ocean_proximity_dummies.columns:
        ocean_proximity_dummies[col] = 0

ocean_proximity_dummies = ocean_proximity_dummies.rename(columns={
    '<1H OCEAN': 'less_than_1h_ocean',
    'INLAND': 'inland',
    'ISLAND': 'island',
    'NEAR BAY': 'near_bay',
    'NEAR OCEAN': 'near_ocean'
})

# Add one-hot encoded columns to merged data
merged_data = pd.concat([merged_data, ocean_proximity_dummies], axis=1)

print("One-hot encoded ocean_proximity columns created:")
print(ocean_proximity_dummies.head())

One-hot encoded ocean_proximity columns created:
   less_than_1h_ocean  inland  island  near_bay  near_ocean
0                   0       0       0         1           0
1                   0       0       0         1           0
2                   0       0       0         1           0
3                   0       0       0         1           0
4                   0       0       0         1           0


In [49]:
# Calculate bedrooms per household with imputation

merged_data['bedrooms_per_household'] = merged_data['total_bedrooms'] / merged_data['households']

postal_code_avg = merged_data.groupby('postal_code')['bedrooms_per_household'].transform('mean')

# Impute missing values with postal code average
merged_data['bedrooms_per_household'] = merged_data['bedrooms_per_household'].fillna(postal_code_avg)

# If still missing (no postal code data), use global average
global_avg = merged_data['bedrooms_per_household'].mean()
merged_data['bedrooms_per_household'] = merged_data['bedrooms_per_household'].fillna(global_avg)

print(f"Bedrooms per household calculated.")
print(f"Missing values after imputation: {merged_data['bedrooms_per_household'].isna().sum()}")
merged_data[['total_bedrooms', 'households', 'bedrooms_per_household', 'postal_code']].head()

Bedrooms per household calculated.
Missing values after imputation: 0


,total_bedrooms,households,bedrooms_per_household,postal_code
0,129.0,126.0,1.023810,94705.0
1,1106.0,1138.0,0.971880,94611.0
2,190.0,177.0,1.073446,94618.0
3,235.0,219.0,1.073059,94618.0
4,280.0,259.0,1.081081,94618.0


In [50]:
# Discretize age into 10-year buckets
def discretize_age(age):
    """Discretize age into 10-year buckets: 0-9, 10-19, 20-29, etc."""
    if pd.isna(age):
        return None
    bucket_start = int(age // 10) * 10
    bucket_end = bucket_start + 9
    return f"{bucket_start}-{bucket_end}"

# Test the function
print("Age discretization examples:")
for age in [5, 15, 25, 35, 45]:
    print(f"Age {age} -> {discretize_age(age)}")

Age discretization examples:
Age 5 -> 0-9
Age 15 -> 10-19
Age 25 -> 20-29
Age 35 -> 30-39
Age 45 -> 40-49


## Aggregate features by neighborhood

In [51]:
# Group by locality-political and aggregate features

# Filter out rows with missing locality-political
merged_data_with_neighborhood = merged_data[merged_data['neighborhood-political'].notna()].copy()


neighborhood_features = merged_data_with_neighborhood.groupby('neighborhood-political').agg({
    'median_house_value': lambda x: min(x.mean(), 500000),
    
    'housing_median_age': 'mean',
    
    'households': lambda x: int(np.ceil(x.mean())),
    
    'bedrooms_per_household': 'mean',
  
    'less_than_1h_ocean': 'max',
    'inland': 'max',
    'island': 'max',
    'near_bay': 'max',
    'near_ocean': 'max'
}).reset_index()

# Rename locality-political to neighborhood
neighborhood_features = neighborhood_features.rename(columns={'neighborhood-political': 'neighborhood'})

# Discretize median house age into 10-year buckets
neighborhood_features['housing_median_age'] = neighborhood_features['housing_median_age'].apply(discretize_age)

# Rename total_households
neighborhood_features = neighborhood_features.rename(columns={'households': 'total_households'})

print(f"Neighborhood features shape: {neighborhood_features.shape}")
print(f"\nFirst few neighborhoods:")
neighborhood_features.head(10)

Neighborhood features shape: (1306, 10)

First few neighborhoods:


,neighborhood,median_house_value,housing_median_age,total_households,bedrooms_per_household,less_than_1h_ocean,inland,island,near_bay,near_ocean
0,28 Palms,95500.000000,40-49,484,1.061983,0,0,0,1,0
1,Acorn Industrial,159800.000000,40-49,366,1.002732,0,0,0,1,0
2,Adams Hill,139050.000000,10-19,89,1.050595,0,1,0,0,0
3,Agua Mansa Industrial Corridor,173900.000000,30-39,256,1.101562,1,0,0,0,0
4,Al Tahoe,125466.666667,20-29,674,1.008520,0,1,0,0,0
5,Alamitos Beach,458350.500000,30-39,261,1.264454,1,0,0,0,0
6,Alessandro,217700.000000,10-19,720,1.011799,1,0,0,0,0
7,Alessandro Heights,137500.000000,40-49,172,1.156977,1,0,0,0,0
8,Alhambra Triangle,116100.000000,30-39,440,1.036364,1,0,0,0,0
9,Alice,243500.000000,20-29,960,1.013542,1,0,0,0,0


In [52]:
# Add event_time feature
import time

current_time_sec = int(round(time.time()))

# Add EventTime feature
neighborhood_features['event_time'] = pd.Series([current_time_sec] * len(neighborhood_features), dtype='float64')

print("\nFinal neighborhood features:")
print(neighborhood_features.head())


Final neighborhood features:
                     neighborhood  median_house_value housing_median_age  \
0                        28 Palms        95500.000000              40-49   
1                Acorn Industrial       159800.000000              40-49   
2                      Adams Hill       139050.000000              10-19   
3  Agua Mansa Industrial Corridor       173900.000000              30-39   
4                        Al Tahoe       125466.666667              20-29   

   total_households  bedrooms_per_household  less_than_1h_ocean  inland  \
0               484                1.061983                   0       0   
1               366                1.002732                   0       0   
2                89                1.050595                   0       1   
3               256                1.101562                   1       0   
4               674                1.008520                   0       1   

   island  near_bay  near_ocean    event_time  
0       0     

## Prepare data for Feature Store

In [53]:

# Cast object dtype to string for Feature Store compatibility
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype('str').astype('string')

cast_object_to_string(neighborhood_features)

# Ensure integer columns are properly typed
neighborhood_features['total_households'] = neighborhood_features['total_households'].astype('int64')
neighborhood_features['less_than_1h_ocean'] = neighborhood_features['less_than_1h_ocean'].astype('int64')
neighborhood_features['inland'] = neighborhood_features['inland'].astype('int64')
neighborhood_features['island'] = neighborhood_features['island'].astype('int64')
neighborhood_features['near_bay'] = neighborhood_features['near_bay'].astype('int64')
neighborhood_features['near_ocean'] = neighborhood_features['near_ocean'].astype('int64')

print("Data types after casting:")
print(neighborhood_features.dtypes)
print("\nSample data:")
neighborhood_features.head()

Data types after casting:
neighborhood              string[python]
median_house_value               float64
housing_median_age        string[python]
total_households                   int64
bedrooms_per_household           float64
less_than_1h_ocean                 int64
inland                             int64
island                             int64
near_bay                           int64
near_ocean                         int64
event_time                       float64
dtype: object

Sample data:


,neighborhood,median_house_value,housing_median_age,total_households,bedrooms_per_household,less_than_1h_ocean,inland,island,near_bay,near_ocean,event_time
0,28 Palms,95500.000000,40-49,484,1.061983,0,0,0,1,0,1.769449e+09
1,Acorn Industrial,159800.000000,40-49,366,1.002732,0,0,0,1,0,1.769449e+09
2,Adams Hill,139050.000000,10-19,89,1.050595,0,1,0,0,0,1.769449e+09
3,Agua Mansa Industrial Corridor,173900.000000,30-39,256,1.101562,1,0,0,0,0,1.769449e+09
4,Al Tahoe,125466.666667,20-29,674,1.008520,0,1,0,0,0,1.769449e+09


## Create Feature Group in SageMaker Feature Store

In [54]:

from sagemaker.feature_store.feature_group import FeatureGroup
from time import gmtime, strftime

neighborhood_feature_group_name = 'neighborhood-feature-group-' + strftime('%d-%H-%M-%S', gmtime())

neighborhood_feature_group = FeatureGroup(
    name=neighborhood_feature_group_name,
    sagemaker_session=feature_store_session
)

print(f"Feature Group Name: {neighborhood_feature_group_name}")

Feature Group Name: neighborhood-feature-group-26-17-35-58


In [55]:
# Load feature definitions
record_identifier_feature_name = 'neighborhood'
event_time_feature_name = 'event_time'

neighborhood_feature_group.load_feature_definitions(data_frame=neighborhood_features)

print("Feature definitions loaded successfully")
print(f"Record Identifier: {record_identifier_feature_name}")
print(f"Event Time Feature: {event_time_feature_name}")

Feature definitions loaded successfully
Record Identifier: neighborhood
Event Time Feature: event_time


In [56]:

def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get('FeatureGroupStatus')
    while status == 'Creating':
        print('Waiting for Feature Group Creation')
        time.sleep(5)
        status = feature_group.describe().get('FeatureGroupStatus')
    if status != 'Created':
        raise RuntimeError(f'Failed to create feature group {feature_group.name}')
    print(f'FeatureGroup {feature_group.name} successfully created.')

neighborhood_feature_group.create(
    s3_uri=f's3://{default_s3_bucket_name}/{prefix}',
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=neighborhood_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup neighborhood-feature-group-26-17-35-58 successfully created.


In [57]:

neighborhood_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:440542329720:feature-group/neighborhood-feature-group-26-17-35-58',
 'FeatureGroupName': 'neighborhood-feature-group-26-17-35-58',
 'RecordIdentifierFeatureName': 'neighborhood',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'neighborhood',
   'FeatureType': 'String'},
  {'FeatureName': 'median_house_value', 'FeatureType': 'Fractional'},
  {'FeatureName': 'housing_median_age', 'FeatureType': 'String'},
  {'FeatureName': 'total_households', 'FeatureType': 'Integral'},
  {'FeatureName': 'bedrooms_per_household', 'FeatureType': 'Fractional'},
  {'FeatureName': 'less_than_1h_ocean', 'FeatureType': 'Integral'},
  {'FeatureName': 'inland', 'FeatureType': 'Integral'},
  {'FeatureName': 'island', 'FeatureType': 'Integral'},
  {'FeatureName': 'near_bay', 'FeatureType': 'Integral'},
  {'FeatureName': 'near_ocean', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'Fractional'}],
 'CreationT

## Ingest Data into Feature Store

In [58]:

print(f"Starting ingestion of {len(neighborhood_features)} records...")
neighborhood_feature_group.ingest(
    data_frame=neighborhood_features, 
    max_workers=3, 
    wait=True
)
print("Data ingestion completed successfully!")

Starting ingestion of 1306 records...
Data ingestion completed successfully!


In [59]:
# confirming data was ingested successfully
sample_neighborhood = neighborhood_features['neighborhood'].iloc[0]
print(f"Testing with sample neighborhood: {sample_neighborhood}")

sample_record = featurestore_runtime.get_record(
    FeatureGroupName=neighborhood_feature_group_name,
    RecordIdentifierValueAsString=sample_neighborhood
)

print("\nSample record retrieved from Feature Store:")
print(sample_record)

Testing with sample neighborhood: 28 Palms

Sample record retrieved from Feature Store:
{'ResponseMetadata': {'RequestId': '34d2440a-96dd-40ca-8d7a-a66cf69df6d8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '34d2440a-96dd-40ca-8d7a-a66cf69df6d8', 'content-type': 'application/json', 'content-length': '922', 'date': 'Mon, 26 Jan 2026 17:36:39 GMT'}, 'RetryAttempts': 0}, 'Record': [{'FeatureName': 'neighborhood', 'ValueAsString': '28 Palms'}, {'FeatureName': 'median_house_value', 'ValueAsString': '95500.0'}, {'FeatureName': 'housing_median_age', 'ValueAsString': '40-49'}, {'FeatureName': 'total_households', 'ValueAsString': '484'}, {'FeatureName': 'bedrooms_per_household', 'ValueAsString': '1.06198347107438'}, {'FeatureName': 'less_than_1h_ocean', 'ValueAsString': '0'}, {'FeatureName': 'inland', 'ValueAsString': '0'}, {'FeatureName': 'island', 'ValueAsString': '0'}, {'FeatureName': 'near_bay', 'ValueAsString': '1'}, {'FeatureName': 'near_ocean', 'ValueAsString': '0'}, {'Fea

## Query Feature Values for Specific Neighborhoods

Required neighborhoods:
1. Brooktree
2. Fisherman's Wharf
3. Los Osos

In [60]:

def display_neighborhood_features(neighborhood_name, record):
    """Display neighborhood features in a readable format"""
    print(f"\n{'='*30}")
    print(f"NEIGHBORHOOD: {neighborhood_name}")
    print(f"{'='*30}")
    
    if 'Record' not in record:
        print("ERROR: Record not found in Feature Store!")
        return
    
    features = record['Record']
    
    # Organize features by category
    print("\nPrimary Key:")
    for feature in features:
        if feature['FeatureName'] == 'neighborhood':
            print(f"  • Neighborhood: {feature['ValueAsString']}")
    
    print("\nEvent Time:")
    for feature in features:
        if feature['FeatureName'] == 'event_time':
            print(f"  • Event Time: {feature['ValueAsString']}")
    
    print("\nOcean Proximity (One-Hot Encoded):")
    for feature in features:
        if feature['FeatureName'] in ['less_than_1h_ocean', 'inland', 'island', 'near_bay', 'near_ocean']:
            label = feature['FeatureName'].replace('_', ' ').title()
            value = "Yes" if feature['ValueAsString'] == '1' else "No"
            print(f"  • {label}: {value}")
    
    print("\nHousing Statistics:")
    for feature in features:
        if feature['FeatureName'] == 'median_house_value':
            print(f"  • Median House Value: ${float(feature['ValueAsString']):,.2f}")
        elif feature['FeatureName'] == 'median_house_age':
            print(f"  • Median House Age: {feature['ValueAsString']} years")
        elif feature['FeatureName'] == 'total_households':
            print(f"  • Total Households: {feature['ValueAsString']}")
        elif feature['FeatureName'] == 'bedrooms_per_household':
            print(f"  • Bedrooms per Household: {float(feature['ValueAsString']):.2f}")
    
    print(f"{'='*30}\n")

In [66]:

print("QUERYING FEATURE STORE FOR: Brooktree")
print("-" * 30)

brooktree_record = featurestore_runtime.get_record(
    FeatureGroupName=neighborhood_feature_group_name,
    RecordIdentifierValueAsString="Brooktree"
)

display_neighborhood_features("Brooktree", brooktree_record)


QUERYING FEATURE STORE FOR: Brooktree
------------------------------

NEIGHBORHOOD: Brooktree

Primary Key:
  • Neighborhood: Brooktree

Event Time:
  • Event Time: 1769448951.0

Ocean Proximity (One-Hot Encoded):
  • Less Than 1H Ocean: No
  • Inland: No
  • Island: No
  • Near Bay: No
  • Near Ocean: Yes

Housing Statistics:
  • Median House Value: $190,600.00
  • Total Households: 243
  • Bedrooms per Household: 1.09



In [67]:

print("QUERYING FEATURE STORE FOR: Fisherman's Wharf")
print("-" * 30)


fishermans_wharf_record = featurestore_runtime.get_record(
    FeatureGroupName=neighborhood_feature_group_name,
    RecordIdentifierValueAsString="Fisherman's Wharf"
)
display_neighborhood_features("Fisherman's Wharf", fishermans_wharf_record)


QUERYING FEATURE STORE FOR: Fisherman's Wharf
------------------------------

NEIGHBORHOOD: Fisherman's Wharf

Primary Key:
  • Neighborhood: Fisherman's Wharf

Event Time:
  • Event Time: 1769448951.0

Ocean Proximity (One-Hot Encoded):
  • Less Than 1H Ocean: No
  • Inland: Yes
  • Island: No
  • Near Bay: No
  • Near Ocean: No

Housing Statistics:
  • Median House Value: $73,800.00
  • Total Households: 363
  • Bedrooms per Household: 0.96



In [68]:

print("QUERYING FEATURE STORE FOR: Los Osos")
print("-" * 80)


los_osos_record = featurestore_runtime.get_record(
    FeatureGroupName=neighborhood_feature_group_name,
    RecordIdentifierValueAsString="Los Osos"
)

display_neighborhood_features("Los Osos", los_osos_record)


QUERYING FEATURE STORE FOR: Los Osos
--------------------------------------------------------------------------------

NEIGHBORHOOD: Los Osos

Primary Key:
  • Neighborhood: Los Osos

Event Time:
  • Event Time: 1769448951.0

Ocean Proximity (One-Hot Encoded):
  • Less Than 1H Ocean: Yes
  • Inland: No
  • Island: No
  • Near Bay: No
  • Near Ocean: No

Housing Statistics:
  • Median House Value: $141,514.29
  • Total Households: 930
  • Bedrooms per Household: 1.03



In [64]:
# List all neighborhoods available in Feature Store
print("Available neighborhoods in the dataset:")
print("-" * 30)
all_neighborhoods = sorted(neighborhood_features['neighborhood'].unique())
print(f"\nTotal neighborhoods: {len(all_neighborhoods)}")
print("\nFirst 20 neighborhoods:")
for i, neighborhood in enumerate(all_neighborhoods[:20], 1):
    print(f"{i:2d}. {neighborhood}")

Available neighborhoods in the dataset:
------------------------------

Total neighborhoods: 1306

First 20 neighborhoods:
 1. 28 Palms
 2. Acorn Industrial
 3. Adams Hill
 4. Agua Mansa Industrial Corridor
 5. Al Tahoe
 6. Alamitos Beach
 7. Alessandro
 8. Alessandro Heights
 9. Alhambra Triangle
10. Alice
11. Allied Gardens
12. Almaden
13. Almaden - Clara Filice
14. Almond Tree
15. Almonte
16. Alta Loma
17. Alta Mesa
18. Alta Vista Homeowners Association
19. Alum Rock
20. Alviso


## Summary of Feature Group

Display a summary of the neighborhood feature group structure and statistics.

In [65]:
# Display Feature Group Summary
print("NEIGHBORHOOD FEATURE GROUP SUMMARY")
print("=" * 30)

print(f"\n Feature Group Name: {neighborhood_feature_group_name}")
print(f" Primary Key: {record_identifier_feature_name}")
print(f" Event Time Feature: {event_time_feature_name}")
print(f"\n Statistics:")
print(f"  • Total neighborhoods ingested: {len(neighborhood_features)}")
print(f"  • Total features per neighborhood: {len(neighborhood_features.columns)}")

print(f"\n Feature List:")
for i, col in enumerate(neighborhood_features.columns, 1):
    dtype = str(neighborhood_features[col].dtype)
    print(f"  {i:2d}. {col:30s} (Type: {dtype})")

print(f"\n Data Quality Metrics:")
print(f"  • Missing values in bedrooms_per_household: {merged_data['bedrooms_per_household'].isna().sum()}")
print(f"  • Median house value cap applied: {(neighborhood_features['median_house_value'] >= 500000).sum()} neighborhoods")
print(f"  • Average households per neighborhood: {neighborhood_features['total_households'].mean():.0f}")

print(f"\n Ocean Proximity Distribution:")
print(f"  • <1H Ocean: {neighborhood_features['less_than_1h_ocean'].sum()} neighborhoods")
print(f"  • Inland: {neighborhood_features['inland'].sum()} neighborhoods")
print(f"  • Island: {neighborhood_features['island'].sum()} neighborhoods")
print(f"  • Near Bay: {neighborhood_features['near_bay'].sum()} neighborhoods")
print(f"  • Near Ocean: {neighborhood_features['near_ocean'].sum()} neighborhoods")

print("\n" + "=" * 30)

NEIGHBORHOOD FEATURE GROUP SUMMARY

 Feature Group Name: neighborhood-feature-group-26-17-35-58
 Primary Key: neighborhood
 Event Time Feature: event_time

 Statistics:
  • Total neighborhoods ingested: 1306
  • Total features per neighborhood: 11

 Feature List:
   1. neighborhood                   (Type: string)
   2. median_house_value             (Type: float64)
   3. housing_median_age             (Type: string)
   4. total_households               (Type: int64)
   5. bedrooms_per_household         (Type: float64)
   6. less_than_1h_ocean             (Type: int64)
   7. inland                         (Type: int64)
   8. island                         (Type: int64)
   9. near_bay                       (Type: int64)
  10. near_ocean                     (Type: int64)
  11. event_time                     (Type: float64)

 Data Quality Metrics:
  • Missing values in bedrooms_per_household: 0
  • Median house value cap applied: 36 neighborhoods
  • Average households per neighborhood: 4

In [1]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>